In [17]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    ISOT
'''



data = pd.read_csv(r"../data/Des_fake_news/LIAR_PROCESSED_train.csv")
test = pd.read_csv(r"../data/Des_fake_news/LIAR_PROCESSED_test.csv")


In [18]:
def multi_flag(sentiment: str):
    if sentiment.lower() == "true":
        return 4
    elif sentiment.lower() == "mostly-true":
        return 3
    elif sentiment.lower() == "half-true":
        return 2
    elif sentiment.lower() == "barely-true":
        return 1
    else:
        return 0
    
data["flag"] = data.apply(lambda x: multi_flag(x["class"]), axis=1)
test["flag"] = test.apply(lambda x: multi_flag(x["class"]), axis=1)   



In [19]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Scale embeddings


#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
filtered = data.dropna()
classes = filtered["flag"].to_numpy()
#features = filtered[["title_vader_class", "text_vader_class", "text_tb_pol_class", "text_tb_sub_class", "title_tb_pol_class", "title_tb_sub_class"]].dropna()

'''
  Tweet classifier
["text_NN_tweets", "title_NN_tweets", "text_log_tweets", title_log_tweets"]
'''

tweet_features = ["text_NN_tweets", "title_NN_tweets", "text_log_tweets", "title_log_tweets", "text_tb_sub_class", "title_tb_sub_class"]
imdb_features = ["text_NN_imdb", "title_NN_imdb", "text_log_imdb", "title_log_imdb"]

tweet_and_imdb = [i for i in tweet_features]
tweet_and_imdb.extend(imdb_features)

features = filtered[["text_tb_pol",	"text_tb_sub",	
              	"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]].to_numpy()


#for i in range(100):
    
x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=42)
#x_train

ensemble = LogisticRegression()
ensemble.fit(x_train, y_train)

# Evaluate predictions
y_pred = ensemble.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic regression, textblob and vader: {accuracy * 100:.2f}%")



uniques, counts = np.unique(y_test, return_counts=True)
counts






Logistic regression, textblob and vader: 27.67%


array([281, 157, 193, 208, 173], dtype=int64)

In [24]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(6, 1)),
  tf.keras.layers.Dense(6, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
'''text only'''
filtered = data.dropna()
classes = filtered["flag"].to_numpy()
#features = filtered[["title_vader_class", "text_vader_class", "text_tb_pol_class", "text_tb_sub_class", "title_tb_pol_class", "title_tb_sub_class"]].to_numpy()
features = filtered[["text_tb_pol",	"text_tb_sub",	
             	"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]].to_numpy()

'''
  Tweet classifier
["text_NN_tweets", "title_NN_tweets", "text_log_tweets", title_log_tweets"]
'''

tweet_features = ["text_NN_tweets", "title_NN_tweets", "text_log_tweets", "title_log_tweets"]
#imdb_features = ["text_NN_imdb", "title_NN_imdb", "text_log_imdb", "title_log_imdb"]

tweet_and_imdb = [i for i in tweet_features]
tweet_and_imdb.extend(imdb_features)

#features = filtered[tweet_and_imdb].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=42)
model.fit(x_train, y_train, epochs=100, verbose=2)




Epoch 1/100


c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


180/180 - 1s - 7ms/step - accuracy: 0.2507 - loss: 1.6199
Epoch 2/100
180/180 - 0s - 2ms/step - accuracy: 0.2558 - loss: 1.6027
Epoch 3/100
180/180 - 0s - 1ms/step - accuracy: 0.2551 - loss: 1.5985
Epoch 4/100
180/180 - 0s - 1ms/step - accuracy: 0.2551 - loss: 1.5968
Epoch 5/100
180/180 - 0s - 1ms/step - accuracy: 0.2527 - loss: 1.5961
Epoch 6/100
180/180 - 0s - 2ms/step - accuracy: 0.2549 - loss: 1.5956
Epoch 7/100
180/180 - 0s - 1ms/step - accuracy: 0.2558 - loss: 1.5950
Epoch 8/100
180/180 - 0s - 1ms/step - accuracy: 0.2528 - loss: 1.5945
Epoch 9/100
180/180 - 0s - 1ms/step - accuracy: 0.2520 - loss: 1.5942
Epoch 10/100
180/180 - 0s - 1ms/step - accuracy: 0.2525 - loss: 1.5941
Epoch 11/100
180/180 - 0s - 2ms/step - accuracy: 0.2525 - loss: 1.5939
Epoch 12/100
180/180 - 0s - 2ms/step - accuracy: 0.2525 - loss: 1.5938
Epoch 13/100
180/180 - 0s - 1ms/step - accuracy: 0.2527 - loss: 1.5938
Epoch 14/100
180/180 - 0s - 1ms/step - accuracy: 0.2539 - loss: 1.5937
Epoch 15/100
180/180 - 0s -

In [ ]:
from sklearn.metrics import confusion_matrix
filtered_2 = test.dropna()


x_true_test = filtered_2[["text_tb_pol",	"text_tb_sub",	
              	"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]].to_numpy()

y_true_vals = filtered_2["flag"].to_numpy()
y_log_pred = ensemble.predict(x_true_test)
accuracy = accuracy_score(y_log_pred, y_true_vals)
print(f"Log LIAR: {accuracy * 100:.2f}%")
results = model.evaluate(x_true_test,  y_true_vals, verbose=2)

matrix = confusion_matrix(y_true_vals, y_log_pred)
matrix


uniques, counts = np.unique(y_true_vals, return_counts=True)

Log LIAR: 24.42%
27/27 - 0s - 3ms/step - accuracy: 0.2431 - loss: 1.6023


0.24305555555555555